In [ ]:
# TWITTER CLUSTERING TASKS WITH KEYWORDS "SHIN TAE YONG"
# START : 2022-10-01, AND END : 2022-10-15

In [ ]:
# RUN THIS FIRST
!pip3 install snscrape

Fungsi **"snscrape"** adalah :
Untuk layanan jejaring sosial (SNS). Seperti mengikis hal-hal seperti profil pengguna, tagar, atau pencarian dan mengembalikan item yang ditemukan.
Contohnya posting yang relevan.

In [ ]:
!pip install langdetect

Fungsi **"langdetect"** adalah untuk mengimplementasi ulang pustaka deteksi bahasa Google ke Python

In [54]:
# THIS PROCESS IS USEFUL FOR IMPORTING THE SNSCRAPE MODULE TO HANDLE THE DATA WE GET FROM TWITTER
import snscrape.modules.twitter as sntwitter
import json
from langdetect import detect

In [55]:
# CHOOSE THE RIGHT KEYWORDS
keywords=['shin tae yong']
start="2022–10–01"
end ="2022–10–15"
max_num=100
fname='tweet.json' 
languages=['id','en']

In [56]:
# CREATE VARIABLE "datatw" FOR, DISPLAY RESULTS OF DATA TAKEN ON TWITTER
import pandas as pd
datatw=[]

Fungsi **Library Panda** : untuk membuat tabel, mengubah dimensi data, mengecek data, dan lain sebagainya. Karena pada Library Panda menyediakan struktur data dan analisis data yang mudah digunakan.


In [57]:
for keyword in keywords:
   
    for i, tweet in enumerate (sntwitter.TwitterSearchScraper(f'{keyword} ').get_items()):
        
        try:
            lan=detect(tweet.content)
        except:
            lan='error'
        if i == max_num:
            break
        if lan in languages:
            data = {'id': tweet.id, 'username':tweet.username, 'date': tweet.date, 'text': tweet.content,'url':tweet.url}
           # print(data)
            datatw.append(tweet.content)
            with open(fname, 'a+', encoding='utf-8') as f:
                line = json.dumps(data, ensure_ascii=False,default=str)
                #print(line)
                f.write(line)
                f.write('\n')

In [ ]:
datatw

In [ ]:
!pip install Sastrawi

Fungsi **Install Sastrawi** : untuk mengurangi kata-kata yang terinfleksi dalam Bahasa Indonesia (Bahasa Indonesia) ke bentuk dasarnya (batang).

In [60]:
# "StopWordRemoverFactory" IS ONE OF THE LIBRARY OF LITERATURE WE HAVE INSTALL
import re
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()# stemming process
# import StopWordRemoverFactory class
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
documents_clean=[]

for d in datatw:
    outputstem= stemmer.stem(d)
    d= stopword.remove(outputstem)
    # Remove Unicode
    document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
    # Remove Mentions
    document_test = re.sub(r'@\w+', '', document_test)
    # Lowercase the document
    document_test = document_test.lower()
    # Remove punctuations
    document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
    # Lowercase the numbers
    document_test = re.sub(r'[0-9]', '', document_test)
    # Remove the doubled space
    outputstop = re.sub(r'\s{2,}', ' ', document_test)
    documents_clean.append(outputstop)

In [ ]:
documents_clean[0:5]

Fungsi **"documents_clean"** adalah : untuk menghapus semua karakter tidak dapat dicetak dari teks

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
tfidfvectorizer = TfidfVectorizer(analyzer='word')
tfidf_wm = tfidfvectorizer.fit_transform(documents_clean)
tfidf_tokens = tfidfvectorizer.get_feature_names()

Fungsi **"TfidfVectorizer"** adalah : sebagai normalisasi nilai bobot yang diperoleh, supaya nilai bobot berada pada range yang sama.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
cv = CountVectorizer()
words = cv.fit_transform(documents_clean)
sum_words = words.sum(axis=0)


words_freq = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()]
words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
frequency = pd.DataFrame(words_freq, columns=['word', 'freq'])

color = plt.cm.twilight(np.linspace(0, 1, 20))
frequency.head(20).plot(x='word', y='freq', kind='bar', figsize=(15, 7), color = color)
plt.title("Most Frequently Occuring Words - Top 20")

Fungsi **"CountVectorizer"** adalah : untuk menghitung frekuensi kata dalam dokumen. Count Vectorizer dapat mengubah fitur teks menjadi sebuah representasi vector.

In [ ]:
# IMPORT K-MEANS FOR CLUSTERING PROCESS
from sklearn.cluster import KMeans
true_k = 3
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(words)

In [ ]:
# DO ORDER CENTROID, TO DISTRIBUTE RESULTS FOR EVERY CLUSTER
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = cv.get_feature_names()

for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

print("\n")

In [ ]:
# DO IT TO SEE THE DENDOGRAM DIAGRAM
import scipy.cluster.hierarchy as sch
X = cv.fit_transform(documents_clean).todense()
dendrogram = sch.dendrogram(sch.linkage(X, method = 'ward',metric='euclidean'),orientation="top")
plt.title('Dendrogram')
plt.xlabel('Jarak Ward')
plt.ylabel('Nomor Dokumen')
plt.show()

Fungsi Method **"ward"** adalah : suatu metode pembentukan cluster yang di dasari oleh hilangnya informasi akibat penggabungan obyek menjadi cluster.

In [ ]:
# DO IT TO SEE THE DENDOGRAM DIAGRAM
import scipy.cluster.hierarchy as sch
X = cv.fit_transform(documents_clean).todense()
dendrogram = sch.dendrogram(sch.linkage(X, method = 'average',metric='euclidean'),orientation="right")
plt.title('Dendrogram')
plt.xlabel('Jarak Rerata')
plt.ylabel('Nomor Dokumen')
plt.show()